# PyZX interoperability: a simple graph

*(This notebook is still being written. It is currently in a "general idea" kind of version, rather than a final version.)*

This notebook offers an example of an algorithmic lattice of a simple PyZX graph. It is helpful for gaining additional insight into the foundational workings of the algorithm and gaining insight into how to prepare and handle a PyZX graph for use with the algorithm.

In [ ]:
import os
import sys
import pyzx as zx

root_folder = os.path.abspath(os.path.join(os.getcwd(), "..", ".."))
if root_folder not in sys.path:
    sys.path.insert(0, root_folder)

from scripts.runner import runner
from utils.interop_pyzx import get_simple_graph_from_pyzx
from utils.utils_zx_graphs import get_zx_type_from_kind

zx.settings.colors = zx.rgb_colors
%matplotlib widget

## Input: Preparing a PyZX graph

Let's start by preparing a PyZX circuit and converting it to a PyZX graph. We'll use a simple graph for this notebook, consisting of three CNOTs. You can also change the circuit freely as long as it is ultimately convertable into a native PyZX graph with all qubit lines interconnected (disconnected qubit lines mean the graph has separate subgraphs / logical computations, in which case the algorithm will only consider one of the subgraphs).

In [ ]:
circuit_name = "pyzx_example_cnots"

c = zx.Circuit(2)
c.add_gate("CNOT", 0, 1)
c.add_gate("CNOT", 1, 0)
c.add_gate("CNOT", 1, 0)

pyzx_graph = c.to_graph()

zx.draw(pyzx_graph, labels=True)

## Process: Running the algorithm

PyZX graphs can have a lot of information that the algorithm does not need to undertake its foundational task of placing spiders in a 3D space.

So, to feed the graph into the algorithm, we first need to convert it into a `simple_graph`, i.e., a simple dictionary of nodes and edges. The conversion removes information that is not needed for foundational operations (ps. while this notebook sticks to the basics, the information can be recovered).

There is a single instruction to distill a PyZX graph into a `simple_graph`. It undertakes all steps below:
- Dump the entire PyZX graph into a dictionary using PyZX internal methods,
- Transform into a dictionary with clear syntax for 3D positioning but all values zeroed out,
- Distill the 3D dictionary into a simpler dictionary for consumption by the algorithm.

In [ ]:
simple_graph = get_simple_graph_from_pyzx(pyzx_graph)
print(simple_graph)

The next step is to feed the `simple_graph` to the algorithm and let it work its magic (it's not magic, but let's pretend it is).

If/when the algorithm is successful, it will produce:
- an interactive visualisation displayed on screen,
- an animation of the process saved to `./outputs/media/`,
- node-by-node/edge-by-edge text results saved to `./outputs/txt/`,
- and related objects for programmatic use.

It is good practice to examine the results printed to screen, too.

In short, the algorithm works by visiting all spiders in a ZX-graph and exchanging them for a 3D version of themselves, while processing edges similarly by encoding them as a special kind of 3D block representing an edge. The printouts on screen detail the order of operations.

Look at the printouts closely.
- The algorithm picks a spider with a high number of edges as starting point and places its corresponding block in a 3D space.
- It then, one by one, places the neighbours of this spider, i.e., all other spiders connected to it.
- When all neighbours of the initial spider are placed, the "current" spider is updated to one of the blocks placed previously.
- Again. 
- And again, 
- Until there are no more spiders left to place.

It is important to highlight that the process above does NOT automatically guarantee all edges in the original ZX graph are handled. For this, after placing all spiders, the algorithm checks for edges that still need to be processed.
- These last set of edges are particularly difficult to clear in 3D because the (source, target) spiders have already been placed in the 3D space. 
- The algorithm needs to figure out a topologically-correct path without changing the position or kind of either block in the (source, target) pair.
- This *often* requires adding intermediary blocks between (source, target), and breaking the edge into several segments.

The final result is therefore a combination of short paths from easy placements and long paths from placements that required intermediary blocks. 

**Note 1.** This notebook limits the number of attempts by the algorithm to a single attempt using the optional `max_attempts=1` parameter. You may need to run the following block a few times to get a successful result. The standard approach is to give the algorithm 10 attempts. The number of attempts is limited here for readability. 

**Note 2.** Each visualisation in the outputs below is an interactive 3D panel. They take a bit to load, but allow you to examine each step in 3D.

In [ ]:
# Define weights for the value function to to choose best of several valid paths per each edge based on: (length of path, number of beams broken by path)
# A negative value for length of path favours short paths.
# A negative value for number of beams broken by path favours placements that do not block potential open faces requiring connections.
VALUE_FUNCTION_HYPERPARAMS = (-1, -0.5)

# Define a desired length of beams
# The longer the beams, the more space between faces. This feature needs improvement but, in theory, increases the odds of success.
# Needs to be combined with an equal or larger `MAX_PATHFINDER_SEARCH_SPACE`
LENGTH_OF_BEAMS = 3

# Define the maximum size of the search space (distance between source and target node for each pathfinding iteration)
# Larger values will produce a larger number of paths, and longer paths. This increases the odds of a successful result, with a significant impact on runtime.
MAX_PATHFINDER_SEARCH_SPACE = 3

kwargs = {
    "weights": VALUE_FUNCTION_HYPERPARAMS,
    "length_of_beams": LENGTH_OF_BEAMS,
    "max_search_space": MAX_PATHFINDER_SEARCH_SPACE,
}

zx.draw(pyzx_graph, labels=True)

simple_graph_after_use, edge_paths, lattice_nodes, lattice_edges = runner(
    simple_graph, circuit_name, strip_boundaries=False, hide_boundaries=False, max_attempts=1, visualise="detail", **kwargs
)

## Output: Using outputs

The algorithm returns four objects:
- `simple_graph_after_use`: the original `simple_graph` returned for convenience.
- `edge_paths`: a set of 3D edges where each edge corresponds to the original edges in the ZX graph but where the "nodes" have been exchanged for all 3D blocks and pipes needed to clear the edge in a 3D space (contains redundant nodes across edges).
- `lattice_nodes`: all nodes in `edge_paths` distilled into a single object with no redundant nodes.
- `lattice_edges`: all edges in `edge_paths` distilled into a single object with no redundant edges.

These objects can be used variously.

For instance, with more complex graphs, it is possible to link information in the original PyZX graph with the results in these objects.

Having said that, since the circuit in this notebook is very simple, we will only visualise how the algorithm constructed each edge. 

This is possible by comparing the original PyZX graph with the `edge_paths` object.

In [ ]:
zx.draw(pyzx_graph, labels=True)

if edge_paths:
    for key, edge in edge_paths.items():
        block_by_block = []
        nodes_and_edges = []
        for node in edge["path_nodes"]:
            block_by_block.append(node[1])
            nodes_and_edges.append(get_zx_type_from_kind(node[1]))
        print(f"{key}: {"-".join(block_by_block)} ({" - ".join(nodes_and_edges)})")